## 3.Drug-Drug Interactions
- The dataset shared in this notebook is a **synthetic dataset** created for demonstration purposes only.
- All results presented in our paper are based on the Merative™ MarketScan® Databases, which contain de-identified real-world healthcare data and cannot be publicly shared.
- For details about the actual maternal and neonatal datasets, please refer to the **Data Availability** and **Methods** sections of our paper.

In [1]:
import os
import sys
import warnings
# Function to set thread limits for external libraries to avoid oversubscription in the shared server
def set_threads_for_external_libraries(n_threads=1):
    if ("numpy" in sys.modules) or ("scipy" in sys.modules) or ("sklearn" in sys.modules):
        warnings.warn("Call set_threads_for_external_libraries() before importing numpy/scipy/sklearn for full effect.")
    for k in ["OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS","VECLIB_MAXIMUM_THREADS","NUMEXPR_NUM_THREADS"]:
        os.environ[k] = str(n_threads)
        
set_threads_for_external_libraries(n_threads=64)

In [2]:
import math
import time
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [3]:
## Load the dataset (This is a synthetic dataset for demonstration purposes. Please read "Data Availability" section, and replace it with your actual dataset.)
df = pd.read_csv('../0_data/synthetic_baby_mom_data.csv')
## make a dataframe for testing
baby_cols_sel = ['RDS_Baby','NAS_Baby','Postmaturity_Baby','ROP_Baby','SGA_Baby']
med_cols_sel = ['Ondansetron, Oral','Sertraline, Oral','Oxycodone, Oral']


In [4]:
## Divide the dataframe into neonatal complications and maternal medications
baby_dz = df[baby_cols_sel]
mom_med = df.loc[:,'Ondansetron, Oral':]
mom_med_count = mom_med.sum()
ga_df = df[['GESTATIONAL_AGE']]

disease_list = baby_dz.columns
ddi_df = pd.concat([baby_dz,mom_med,ga_df],axis=1)

In [5]:
## Calculate Drug-Drug Interaction effects on Neonatal Complications
d_final={}
j=0 ## Index number (valid indices)
i=0 ## Number of pairs

for DISEASE in disease_list:
    for med1 in mom_med.columns:
        med1_ix = mom_med.columns.get_loc(med1)
        for med2 in mom_med.columns[med1_ix+1:]:
            sel_df = ddi_df[[DISEASE,med1,med2,'GESTATIONAL_AGE']]
            sel_df['Interaction'] = sel_df[med1]*sel_df[med2]
            med12_count = sel_df['Interaction'].sum()
            if med12_count==0:
                pass
            elif ((sel_df['Interaction']!=0)&(sel_df[DISEASE]!=0)).sum()<=10:
                pass
            else:
                stats_model_input = '{} ~ sel_df.iloc[:,1] + sel_df.iloc[:,2] + Interaction + GESTATIONAL_AGE'.format(DISEASE)
                try:
                    reg = smf.logit(stats_model_input, data=sel_df).fit()
                    b1= reg.params['sel_df.iloc[:, 1]']
                    b2=reg.params['sel_df.iloc[:, 2]']
                    b3=reg.params['Interaction']
                    b1_pval = reg.pvalues['sel_df.iloc[:, 1]']
                    b2_pval = reg.pvalues['sel_df.iloc[:, 2]']
                    b3_pval=reg.pvalues['Interaction']
                except:
                    b1= np.nan
                    b2= np.nan
                    b3= np.nan
                    b1_pval = np.nan
                    b2_pval = np.nan
                    b3_pval= np.nan
                j+=1
                dz_med12_count = ((sel_df['Interaction']!=0)&(sel_df[DISEASE]!=0)).sum()
                med1_count = mom_med_count[med1]
                med2_count = mom_med_count[med2]
                d_final[j]={'Disease':DISEASE,'Med1':med1,'Med2':med2,'b1':b1,'b2':b2,'b3':b3,'pval(b1)':b1_pval,'pval(b2)':b2_pval,'pval(b3)':b3_pval,'Med1(count)':med1_count,'Med2(count)':med2_count,'Med1/2(count)':med12_count,'Dz+Med1/2(count)':dz_med12_count} 
                print(d_final[j])
            i+=1

/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]


Optimization terminated successfully.
         Current function value: 0.139557
         Iterations 8
{'Disease': 'RDS_Baby', 'Med1': 'Ondansetron, Oral', 'Med2': 'Sertraline, Oral', 'b1': 1.7147293012618319, 'b2': 0.043053308986753636, 'b3': 0.08183670302157744, 'pval(b1)': 0.0, 'pval(b2)': 0.5517103325948738, 'pval(b3)': 0.7444869641369483, 'Med1(count)': 8650, 'Med2(count)': 6530, 'Med1/2(count)': 129, 'Dz+Med1/2(count)': 21}


/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]
/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]


Optimization terminated successfully.
         Current function value: 0.139554
         Iterations 8
{'Disease': 'RDS_Baby', 'Med1': 'Ondansetron, Oral', 'Med2': 'Acetaminophen, Oral', 'b1': 1.7275337535370674, 'b2': 0.04137076651840976, 'b3': -0.32209775797568346, 'pval(b1)': 0.0, 'pval(b2)': 0.3122195902875362, 'pval(b3)': 0.07351880089034712, 'Med1(count)': 8650, 'Med2(count)': 21000, 'Med1/2(count)': 328, 'Dz+Med1/2(count)': 38}


/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]
/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]
/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

Optimization terminated successfully.
         Current function value: 0.136059
         Iterations 7
{'Disease': 'Postmaturity_Baby', 'Med1': 'Ondansetron, Oral', 'Med2': 'Acetaminophen, Oral', 'b1': -0.040036615059366426, 'b2': 0.03314028163973638, 'b3': 0.11003221745847734, 'pval(b1)': 0.5420206988472285, 'pval(b2)': 0.41673814148260235, 'pval(b3)': 0.7278349267663017, 'Med1(count)': 8650, 'Med2(count)': 21000, 'Med1/2(count)': 328, 'Dz+Med1/2(count)': 11}


/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]
/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['Interaction'] = sel_df[med1]*sel_df[med2]
/tmp/ipykernel_993860/3777853673.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [6]:
## convert the final dictionary to a dataframe
ddi_table = pd.DataFrame.from_dict(d_final,'index')
ddi_table

,Disease,Med1,Med2,b1,b2,b3,pval(b1),pval(b2),pval(b3),Med1(count),Med2(count),Med1/2(count),Dz+Med1/2(count)
1,RDS_Baby,"Ondansetron, Oral","Sertraline, Oral",1.714729,0.043053,0.081837,0.000000,0.551710,0.744487,8650,6530,129,21
2,RDS_Baby,"Ondansetron, Oral","Acetaminophen, Oral",1.727534,0.041371,-0.322098,0.000000,0.312220,0.073519,8650,21000,328,38
3,Postmaturity_Baby,"Ondansetron, Oral","Acetaminophen, Oral",-0.040037,0.033140,0.110032,0.542021,0.416738,0.727835,8650,21000,328,11


In [7]:
def Benjamini_Hochberg_correction(df, p_value_column,p_value=0.05):
    """
    Benjamini-Hochberg correction for multiple hypothesis testing.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing p-values.
    p_value_column : str
        Column name of p-values.
    q_value_column : str
        Column name of q-values.
        
    Returns
    -------
    None
    """
    df = df.sort_values(by=p_value_column, ascending=True)
    df = df.reset_index().drop(columns=['index'],axis=1)
    df['k']=df.index+1
    df['m']=df.shape[0]
    df['a']=0.05
    df['B-H critical value']=df['k']*df['a']/df['m']
    df['BH-significance']=(df[p_value_column]<df['B-H critical value'])
    BH_true_df = df[df['BH-significance']==True]
    return BH_true_df

In [8]:
Benjamini_Hochberg_correction(ddi_table, 'pval(b3)',p_value=0.05)

,Disease,Med1,Med2,b1,b2,b3,pval(b1),pval(b2),pval(b3),Med1(count),Med2(count),Med1/2(count),Dz+Med1/2(count),k,m,a,B-H critical value,BH-significance
